In [93]:
import time
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
from supabase import create_client

In [102]:
# Connect to database
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
client = create_client(url, key)

email = 'jackson.shinpaugh@gmail.com'
password = 'Zestymeatloaf1'
user = client.auth.sign_in(email=email, password=password)

In [103]:
# All possible entries
all_possible_annotations = ['brawling', 'br', 'restraining', 're', 'pepper spray', 'ps', 'striking', 
                            'st', 'advancing', 'ad', 'crowd', 'cr', 'running', 'ru', 'person on ground', 'pg', 
                            'blood', 'bl', 'throwing', 'th', 'aiming', 'am', 'pointing', 'po', 'nothing', 'no']

value_pairs = {'br': 'brawling', 're': 'restraining', 'ps': 'spray', 'st': 'striking', 'ad': 'advancing', 
                'cr': 'crowd', 'ru': 'running', 'pg': 'person on ground', 'bl': 'blood', 
                'th': 'throwing', 'am': 'aiming', 'po': 'person pointing', 'nothing': 'no'} 

clarity_options = ['easy', 'medium', 'hard']

In [105]:
# enter user id
user_id = input('enter user id: ')

# begin application that updates database
while True:
   
    vid_url = input("enter full url of youtube video, or enter d to save & exit")
    list_url = vid_url.split('&list')
    newvid_url = list_url[0]

    # check if url is already in table
    existing_urls = list(client.table('urls').select('url').execute())[0][1]
    url_search = next((item for item in existing_urls if item["url"] == newvid_url), None)
    if url_search is not None:
        print('URL ALREADY USED')
        continue

    # generate key for dict
    vid_strs = newvid_url.split("=")
    vid_key = vid_strs[-1]
    
    if newvid_url == 'd': break
    
    # update urls table with new url
    client.table('urls').insert({'url': newvid_url, 'youtube_id': vid_key}).execute()

    if newvid_url == '': continue

    time.sleep(1)
    print(f"Current video: {newvid_url}")

    i = 1
    

    # Begin collecting annotations for the video
    while True:
        user_annotation = input("Add annotation (label), or enter 'x' to delete an annot, enter 'd' if done with video")
        
        if newvid_url == '': continue
        if user_annotation == "d": break
        
        clarity_level = input('Input clarity level (easy, medium, hard)')
        if clarity_level not in clarity_options:
            print('INCORRECT ENTRY - RETRY')
            continue
        
        #create unique key id
        new_vid_key = vid_key + str(i)
        
        #if label is valid
        if user_annotation in all_possible_annotations:
            print(f'annot_{i}: {user_annotation}')
            
            user_segment = input("enter segment (ex: 1:12, 1:20)")
            print(f"annot_{i} segment: {user_segment}")

            if user_segment == '': continue

            try:
                #convert time to total seconds
                user_segment = user_segment.replace(",", "")
                time_list = user_segment.split(' ')
                time_list = [item.split(":") for item in time_list]
                time_list[0][0] = float(time_list[0][0]) * 60
                time_list[1][0] = float(time_list[1][0]) * 60
                time_list[0][1] = float(time_list[0][1])
                time_list[1][1] = float(time_list[1][1])
                time_listl = time_list[0][0] + float(time_list[0][1])
                time_listr = time_list[1][0] + float(time_list[1][1])

                #convert label abreviations to full word
                if user_annotation in value_pairs.keys():
                    user_annotation = value_pairs[user_annotation]
                
                # fill in with sql queries to update var_data table
                client.table('var_data').insert({'id': new_vid_key, 'label': user_annotation, 
                                                'time_start': time_listl, 'time_end': time_listr,
                                                'clarity_level': clarity_level, 'youtube_id': vid_key,
                                                'user_id': user_id}).execute()
                
                i += 1
            
            except:
                print("INCORRECT ENTRY - RETRYING")
                continue
        
        elif user_annotation == "x":
            annot_to_delete = input("what annot do you want to delete? (ex: annot_1), Or enter x to cancel")

            if annot_to_delete == '': continue

            if annot_to_delete == 'x': continue

            else:
                print(f'{annot_to_delete}, DELETED')
                ind = annot_to_delete[-1:]
                # delete row from var_data table
                client.table('var_data').delete().eq('id', vid_key+ind).execute()               
                continue

        else:
            continue   


URL ALREADY USED
